## libraries

In [302]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.preprocessing import LabelEncoder
from src.util import catgvssale,contvssale,contvscont,check_column_skewness,remove_skewness,plot_contv,remove_ngskewness
from datetime import datetime
from scipy.stats import skew
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectPercentile, chi2
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import pickle

In [3]:
final_df = pd.read_csv('data/raw/train.csv')
df1 =  pd.read_csv('data/raw/train.csv')
lbl_dsc = pd.read_csv('data\clean\label_description.csv')
var_dsc =  pd.read_csv('data/clean/variable_description.csv')

In [4]:
var_dsc

,variable,variable_type,variable_description
0,MSSubClass,Nominal,Identifies the type of dwelling involved in t...
1,MSZoning,Nominal,Identifies the general zoning classification ...
2,LotFrontage,Continuous,Linear feet of street connected to property
3,LotArea,Continuous,Lot size in square feet
4,Street,Nominal,Type of road access to property
...,...,...,...
75,MoSold,Discrete,Month Sold (MM)
76,YrSold,Discrete,Year Sold (YYYY)
77,SaleType,Nominal,Type of sale
78,SaleCondition,Nominal,Condition of sale


In [5]:
nominal_df = var_dsc.loc[var_dsc['variable_type'] == 'Nominal']
#print(ordinal_df)
nominal_df.head(30)

,variable,variable_type,variable_description
0,MSSubClass,Nominal,Identifies the type of dwelling involved in t...
1,MSZoning,Nominal,Identifies the general zoning classification ...
4,Street,Nominal,Type of road access to property
5,Alley,Nominal,Type of alley access to property
7,LandContour,Nominal,Flatness of the property
9,LotConfig,Nominal,Lot configuration
11,Neighborhood,Nominal,Physical locations within Ames city limits
12,Condition1,Nominal,Proximity to various conditions
13,Condition2,Nominal,Proximity to various conditions (if more than...
14,BldgType,Nominal,Type of dwelling


In [6]:
ordinal_df = var_dsc.loc[var_dsc['variable_type'] == 'Ordinal']
#print(ordinal_df)
ordinal_df.head(24)

,variable,variable_type,variable_description
6,LotShape,Ordinal,General shape of property
8,Utilities,Ordinal,Type of utilities available
10,LandSlope,Ordinal,Slope of property
16,OverallQual,Ordinal,Rates the overall material and finish of the ...
17,OverallCond,Ordinal,Rates the overall condition of the house
26,ExterQual,Ordinal,Evaluates the quality of the material on the ...
27,ExterCond,Ordinal,Evaluates the present condition of the materi...
29,BsmtQual,Ordinal,Evaluates the height of the basement
30,BsmtCond,Ordinal,Evaluates the general condition of the basement
31,BsmtExposure,Ordinal,Refers to walkout or garden level walls


In [7]:
cont_df = var_dsc.loc[var_dsc['variable_type'] == 'Continuous']
cont_df

,variable,variable_type,variable_description
2,LotFrontage,Continuous,Linear feet of street connected to property
3,LotArea,Continuous,Lot size in square feet
25,MasVnrArea,Continuous,Masonry veneer area in square feet
33,BsmtFinSF1,Continuous,Type 1 finished square feet
35,BsmtFinSF2,Continuous,Type 2 finished square feet
36,BsmtUnfSF,Continuous,Unfinished square feet of basement area
37,TotalBsmtSF,Continuous,Total square feet of basement area
42,1stFlrSF,Continuous,First Floor square feet
43,2ndFlrSF,Continuous,Second floor square feet
44,LowQualFinSF,Continuous,Low quality finished square feet (all floors)


In [8]:
final_df

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


In [9]:
cont_list = cont_df['variable'].to_list()
def main_fun(data,cont_list):
    
    for i in cont_list:
        column_values = data[i]
        skewness = column_values.skew()
        if abs(skewness) > 1:
            
            column_values = np.log1p(column_values)

        
        data[i] = column_values
    return data 

#cont_list = cont_df['variable'].to_list()
main_fun(final_df, cont_list)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,4.189655,9.042040,Pave,NaN,Reg,Lvl,AllPub,...,0.0,NaN,NaN,NaN,0.000000,2,2008,WD,Normal,12.247699
1,2,20,RL,4.394449,9.169623,Pave,NaN,Reg,Lvl,AllPub,...,0.0,NaN,NaN,NaN,0.000000,5,2007,WD,Normal,12.109016
2,3,60,RL,4.234107,9.328212,Pave,NaN,IR1,Lvl,AllPub,...,0.0,NaN,NaN,NaN,0.000000,9,2008,WD,Normal,12.317171
3,4,70,RL,4.110874,9.164401,Pave,NaN,IR1,Lvl,AllPub,...,0.0,NaN,NaN,NaN,0.000000,2,2006,WD,Abnorml,11.849405
4,5,60,RL,4.442651,9.565284,Pave,NaN,IR1,Lvl,AllPub,...,0.0,NaN,NaN,NaN,0.000000,12,2008,WD,Normal,12.429220
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,4.143135,8.976894,Pave,NaN,Reg,Lvl,AllPub,...,0.0,NaN,NaN,NaN,0.000000,8,2007,WD,Normal,12.072547
1456,1457,20,RL,4.454347,9.486152,Pave,NaN,Reg,Lvl,AllPub,...,0.0,NaN,MnPrv,NaN,0.000000,2,2010,WD,Normal,12.254868
1457,1458,70,RL,4.204693,9.109746,Pave,NaN,Reg,Lvl,AllPub,...,0.0,NaN,GdPrv,Shed,7.824446,5,2010,WD,Normal,12.493133
1458,1459,20,RL,4.234107,9.181735,Pave,NaN,Reg,Lvl,AllPub,...,0.0,NaN,NaN,NaN,0.000000,4,2010,WD,Normal,11.864469


## columns to drop 

In [10]:
column_to_drop = ['PoolQC','MiscFeature','Alley','Fence']
#final_df = final_df.drop(['MiscFeature','Alley','Fence'],axis=1)
final_df

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,4.189655,9.042040,Pave,NaN,Reg,Lvl,AllPub,...,0.0,NaN,NaN,NaN,0.000000,2,2008,WD,Normal,12.247699
1,2,20,RL,4.394449,9.169623,Pave,NaN,Reg,Lvl,AllPub,...,0.0,NaN,NaN,NaN,0.000000,5,2007,WD,Normal,12.109016
2,3,60,RL,4.234107,9.328212,Pave,NaN,IR1,Lvl,AllPub,...,0.0,NaN,NaN,NaN,0.000000,9,2008,WD,Normal,12.317171
3,4,70,RL,4.110874,9.164401,Pave,NaN,IR1,Lvl,AllPub,...,0.0,NaN,NaN,NaN,0.000000,2,2006,WD,Abnorml,11.849405
4,5,60,RL,4.442651,9.565284,Pave,NaN,IR1,Lvl,AllPub,...,0.0,NaN,NaN,NaN,0.000000,12,2008,WD,Normal,12.429220
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,4.143135,8.976894,Pave,NaN,Reg,Lvl,AllPub,...,0.0,NaN,NaN,NaN,0.000000,8,2007,WD,Normal,12.072547
1456,1457,20,RL,4.454347,9.486152,Pave,NaN,Reg,Lvl,AllPub,...,0.0,NaN,MnPrv,NaN,0.000000,2,2010,WD,Normal,12.254868
1457,1458,70,RL,4.204693,9.109746,Pave,NaN,Reg,Lvl,AllPub,...,0.0,NaN,GdPrv,Shed,7.824446,5,2010,WD,Normal,12.493133
1458,1459,20,RL,4.234107,9.181735,Pave,NaN,Reg,Lvl,AllPub,...,0.0,NaN,NaN,NaN,0.000000,4,2010,WD,Normal,11.864469


In [11]:
#ordinal_df = ordinal_df.drop(['PoolQC'],axis=0)
ordinal_df

,variable,variable_type,variable_description
6,LotShape,Ordinal,General shape of property
8,Utilities,Ordinal,Type of utilities available
10,LandSlope,Ordinal,Slope of property
16,OverallQual,Ordinal,Rates the overall material and finish of the ...
17,OverallCond,Ordinal,Rates the overall condition of the house
26,ExterQual,Ordinal,Evaluates the quality of the material on the ...
27,ExterCond,Ordinal,Evaluates the present condition of the materi...
29,BsmtQual,Ordinal,Evaluates the height of the basement
30,BsmtCond,Ordinal,Evaluates the general condition of the basement
31,BsmtExposure,Ordinal,Refers to walkout or garden level walls


In [12]:
ordinal_list = ordinal_df['variable'].to_list()
print(ordinal_list)
l = len(ordinal_list)
print(l)
ordinal_list.remove("PoolQC")
ordinal_list.remove("Fence")
print(ordinal_list)

['LotShape', 'Utilities', 'LandSlope', 'OverallQual', 'OverallCond', 'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'HeatingQC', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence']
23
['LotShape', 'Utilities', 'LandSlope', 'OverallQual', 'OverallCond', 'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'HeatingQC', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive']


In [13]:
cont_list = cont_df['variable'].to_list()
print(cont_list)

['LotFrontage', 'LotArea', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'SalePrice']


In [14]:
nominal_list = nominal_df['variable'].to_list()
print(nominal_list)
nominal_list.remove("MiscFeature")
nominal_list.remove("Alley")
nominal_list.remove("MSSubClass")
print(nominal_list)

['MSSubClass', 'MSZoning', 'Street', 'Alley', 'LandContour', 'LotConfig', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'Foundation', 'Heating', 'CentralAir', 'GarageType', 'MiscFeature', 'SaleType', 'SaleCondition']
['MSZoning', 'Street', 'LandContour', 'LotConfig', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'Foundation', 'Heating', 'CentralAir', 'GarageType', 'SaleType', 'SaleCondition']


In [15]:
notnum_list = cont_list + ordinal_list

print(notnum_list)


['LotFrontage', 'LotArea', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'SalePrice', 'LotShape', 'Utilities', 'LandSlope', 'OverallQual', 'OverallCond', 'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'HeatingQC', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive']


In [16]:
ordinal_vars = ordinal_list
continuous_vars = cont_list
nominal_vars =  nominal_list
comb_var = notnum_list

In [17]:
df1

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


In [18]:
df1 = df1.drop(['Alley','PoolQC','MiscFeature','Fence'],axis=1)
df1

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,...,0,0,0,0,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,...,0,0,0,0,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,...,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,...,0,0,0,0,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,Reg,Lvl,AllPub,Inside,...,112,0,0,0,0,4,2010,WD,Normal,142125


154500

In [19]:
df1.fillna(0)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,...,0,0,0,0,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,...,0,0,0,0,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,...,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,...,0,0,0,0,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,Reg,Lvl,AllPub,Inside,...,112,0,0,0,0,4,2010,WD,Normal,142125


## labelencoding for ordinal data types

In [20]:
def label_encode_dataset(dataset, ordinal_vars):
    label_encoder = LabelEncoder()

    for col in ordinal_vars:
        dataset[col] = label_encoder.fit_transform(dataset[col])

    return dataset

encoded_data = label_encode_dataset(df1, ordinal_vars)
print(encoded_data)


        Id  MSSubClass MSZoning  LotFrontage  LotArea Street  LotShape   
0        1          60       RL         65.0     8450   Pave         3  \
1        2          20       RL         80.0     9600   Pave         3   
2        3          60       RL         68.0    11250   Pave         0   
3        4          70       RL         60.0     9550   Pave         0   
4        5          60       RL         84.0    14260   Pave         0   
...    ...         ...      ...          ...      ...    ...       ...   
1455  1456          60       RL         62.0     7917   Pave         3   
1456  1457          20       RL         85.0    13175   Pave         3   
1457  1458          70       RL         66.0     9042   Pave         3   
1458  1459          20       RL         68.0     9717   Pave         3   
1459  1460          20       RL         75.0     9937   Pave         3   

     LandContour  Utilities LotConfig  ...  EnclosedPorch 3SsnPorch   
0            Lvl          0    Inside  .

## one hot encoding for nominal data 

In [21]:


def one_hot_encode_nominal(dataset, nominal_vars):
    encoded_dataset = dataset.copy()

    for var in nominal_vars:
        # Perform one-hot encoding
        encoded_columns = pd.get_dummies(encoded_dataset[var], prefix=var, drop_first=True)
        encoded_columns = encoded_columns.astype(int)
        encoded_dataset = pd.concat([encoded_dataset, encoded_columns], axis=1)
        encoded_dataset.drop(columns=var,inplace=True)
        
    return encoded_dataset


encoded_data = one_hot_encode_nominal(encoded_data, nominal_vars)
print(encoded_data)



        Id  MSSubClass  LotFrontage  LotArea  LotShape  Utilities  LandSlope   
0        1          60         65.0     8450         3          0          0  \
1        2          20         80.0     9600         3          0          0   
2        3          60         68.0    11250         0          0          0   
3        4          70         60.0     9550         0          0          0   
4        5          60         84.0    14260         0          0          0   
...    ...         ...          ...      ...       ...        ...        ...   
1455  1456          60         62.0     7917         3          0          0   
1456  1457          20         85.0    13175         3          0          0   
1457  1458          70         66.0     9042         3          0          0   
1458  1459          20         68.0     9717         3          0          0   
1459  1460          20         75.0     9937         3          0          0   

      OverallQual  OverallCond  YearBui

In [22]:
temp = one_hot_encode_nominal(encoded_data,ordinal_vars)
print(temp)

        Id  MSSubClass  LotFrontage  LotArea  YearBuilt  YearRemodAdd   
0        1          60         65.0     8450       2003          2003  \
1        2          20         80.0     9600       1976          1976   
2        3          60         68.0    11250       2001          2002   
3        4          70         60.0     9550       1915          1970   
4        5          60         84.0    14260       2000          2000   
...    ...         ...          ...      ...        ...           ...   
1455  1456          60         62.0     7917       1999          2000   
1456  1457          20         85.0    13175       1978          1988   
1457  1458          70         66.0     9042       1941          2006   
1458  1459          20         68.0     9717       1950          1996   
1459  1460          20         75.0     9937       1965          1965   

      MasVnrArea  BsmtFinSF1  BsmtFinSF2  BsmtUnfSF  ...  GarageQual_3   
0          196.0         706           0        1

## log transform for continuous variables

In [23]:


def log_transform_continuous(dataset, continuous_vars):
    transformed_dataset = dataset.copy()

    for var in continuous_vars:
        # Compute skewness
        skewness = skew(transformed_dataset[var])

        if skewness > 1:
            # Perform log transform
            transformed_dataset[var] = np.log1p(transformed_dataset[var])

    return transformed_dataset



final_data = log_transform_continuous(temp, continuous_vars)
print(final_data)


        Id  MSSubClass  LotFrontage   LotArea  YearBuilt  YearRemodAdd   
0        1          60         65.0  9.042040       2003          2003  \
1        2          20         80.0  9.169623       1976          1976   
2        3          60         68.0  9.328212       2001          2002   
3        4          70         60.0  9.164401       1915          1970   
4        5          60         84.0  9.565284       2000          2000   
...    ...         ...          ...       ...        ...           ...   
1455  1456          60         62.0  8.976894       1999          2000   
1456  1457          20         85.0  9.486152       1978          1988   
1457  1458          70         66.0  9.109746       1941          2006   
1458  1459          20         68.0  9.181735       1950          1996   
1459  1460          20         75.0  9.204121       1965          1965   

      MasVnrArea  BsmtFinSF1  BsmtFinSF2  BsmtUnfSF  ...  GarageQual_3   
0          196.0    6.561031    0.000

In [48]:
columnindex = final_data.columns.get_loc('SalePrice')
print(columnindex)

35


## splitting the data

In [53]:
X1 = final_data.iloc[:,0:35]
y = final_data.iloc[:,35]
X2 = final_data.iloc[:,36:262]
X = pd.concat([X1,X2],axis=1)
X = X.fillna(0)
X


,Id,MSSubClass,LotFrontage,LotArea,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,...,GarageQual_3,GarageQual_4,GarageQual_5,GarageCond_1,GarageCond_2,GarageCond_3,GarageCond_4,GarageCond_5,PavedDrive_1,PavedDrive_2
0,1,60,65.0,9.042040,2003,2003,196.0,6.561031,0.000000,150,...,0,1,0,0,0,0,1,0,0,1
1,2,20,80.0,9.169623,1976,1976,0.0,6.886532,0.000000,284,...,0,1,0,0,0,0,1,0,0,1
2,3,60,68.0,9.328212,2001,2002,162.0,6.188264,0.000000,434,...,0,1,0,0,0,0,1,0,0,1
3,4,70,60.0,9.164401,1915,1970,0.0,5.379897,0.000000,540,...,0,1,0,0,0,0,1,0,0,1
4,5,60,84.0,9.565284,2000,2000,350.0,6.486161,0.000000,490,...,0,1,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,62.0,8.976894,1999,2000,0.0,0.000000,0.000000,953,...,0,1,0,0,0,0,1,0,0,1
1456,1457,20,85.0,9.486152,1978,1988,119.0,6.673298,5.099866,589,...,0,1,0,0,0,0,1,0,0,1
1457,1458,70,66.0,9.109746,1941,2006,0.0,5.620401,0.000000,877,...,0,1,0,0,0,0,1,0,0,1
1458,1459,20,68.0,9.181735,1950,1996,0.0,3.912023,6.937314,0,...,0,1,0,0,0,0,1,0,0,1


In [54]:
y

0       12.247699
1       12.109016
2       12.317171
3       11.849405
4       12.429220
          ...    
1455    12.072547
1456    12.254868
1457    12.493133
1458    11.864469
1459    11.901590
Name: SalePrice, Length: 1460, dtype: float64

## linear regression with the mutual info data

In [284]:
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SelectKBest, mutual_info_regression
from sklearn.model_selection import train_test_split
# Split the data into features (X) and target variable (y)

# Apply SelectKBest feature selection
k = 200 # Number of top features to select
selector = SelectKBest(score_func=mutual_info_regression, k=k)
X_selected = selector.fit_transform(X, y)

X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.3, random_state=42)


# Get selected feature names
#selected_feature_names = X.columns[selector.get_support()].tolist()

# Fit a linear regression model using the selected features
linear_regression = LinearRegression()
linear_regression.fit(X_train, y_train)


LinearRegression()

save the model

In [303]:
#filename = 'lineareg_modelexp.sav'
#pickle.dump(linear_regression, open(filename, 'wb'))

In [285]:
y_pred = linear_regression.predict(X_test)


In [286]:
y_predsp = np.expm1(y_pred)
y_predsp

array([147598.2371021 , 341603.75685533, 107367.20960213, 175508.05842846,
       331704.34238876,  78280.29487728, 256145.11609984, 140028.951376  ,
        75178.98690233, 138907.47428428, 154015.57059601, 115537.06245072,
        62697.23125715, 203893.03365192, 178952.01798951, 134562.06511627,
       190273.28727361, 134781.57604561, 103580.29240034, 212035.10173098,
       172871.43013892, 210727.93679376, 180513.62107348, 132098.41122003,
       199603.28503749, 160161.13539019, 193205.80730897, 113978.79378832,
       177457.39143389, 204323.94273134, 171776.77981872, 268460.35024351,
       200520.0245124 , 118819.56279951, 257087.46731777, 143656.33623129,
       155187.0968199 , 198756.38536302, 352887.72936092, 107124.19904094,
       121672.43133768, 232493.75114931, 110837.78336316, 326524.65709188,
       129656.89000557, 139049.47371548, 107276.6282212 , 127849.00316347,
       439614.43654969, 139538.04050217, 114884.59311797, 197223.66131782,
       102061.76467374, 3

In [287]:
y_testsp = np.expm1(y_test)
#print(sp)

In [288]:
df1['SalePrice'].iloc[892]

154500

In [289]:
mse = mean_squared_error(y_testsp, y_predsp)
r2 = r2_score(y_testsp, y_predsp)
rmse = mean_squared_error(y_testsp, y_predsp,squared=False)
print("Mean Squared Error (RMSE) on train set:", mse)
print("Root Mean Squared Error (MSE) on train set:", rmse)
print("R-squared score on train set:", r2)

Mean Squared Error (RMSE) on train set: 527304267.00082177
R-squared score on train set: 0.9244343390809008
Root Mean Squared Error (MSE) on train set: 22963.106649598216


In [290]:
le = round(linear_regression.score(X_test,y_test), 2)
print(le)

0.91


with k = 50.mse = 3.408924337252029e-32 on the test data.smallest value of mse.

with k = 100 mse = 1.6952560948258837e-29 on the test data.

with k = 200 mse = 3.3458167857940476e-28 on the test data. larger value of mse

with k = 261 mse = 6.126583433723093e-28 on the test data

## performing cross validation to reduce the rmse value

In [311]:
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression


In [312]:
k_folds = 5
kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)


In [316]:
model = LinearRegression()
scores = cross_val_score(model, X_selected, y, scoring='neg_mean_squared_error', cv=kf)
print(scores)

[-0.01539813 -0.01542323 -0.03011461 -0.01908196 -0.01311282]


## randomforest Regression

In [305]:
from sklearn.ensemble import RandomForestRegressor
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.3, random_state=42)
# create regressor object
regressor = RandomForestRegressor(n_estimators=100,random_state=0,oob_score=True)
 
# fit the regressor with x and y data
regressor.fit(X_train, y_train)

RandomForestRegressor(oob_score=True, random_state=0)

In [306]:
Y_pred = regressor.predict(X_test)

In [308]:
rg = round(regressor.score(X_test,y_test))
print(rg)

1


In [310]:
print(regressor.oob_score_)

0.851284275641637


In [309]:
r2 = r2_score(y_testsp, y_predsp)
rmse = mean_squared_error(y_testsp, y_predsp,squared=False)
print("Mean Squared Error (RMSE) on train set:", mse)
print("Root Mean Squared Error (MSE) on train set:", rmse)

Mean Squared Error (RMSE) on train set: 0.06983288204850825
Root Mean Squared Error (MSE) on train set: 22963.106649598216


## model Linear Regression

In [291]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [292]:
linear_regression = LinearRegression()
linear_regression.fit(X_train, y_train)

LinearRegression()

In [293]:
y_pred = linear_regression.predict(X_test)
y_pred

array([11.91481616, 12.76613287, 11.56418412, 12.07079985, 12.68400588,
       11.28746836, 12.4664232 , 11.90846221, 11.23964036, 11.8745827 ,
       11.90299682, 11.66438582, 11.24380333, 12.23099565, 12.06179502,
       11.79401072, 12.14911208, 11.80178724, 11.60378094, 12.27595368,
       11.99462724, 12.26655372, 12.05240825, 11.79458497, 12.1948426 ,
       11.94518918, 12.16738666, 11.65897167, 12.10846911, 12.18048985,
       11.95237354, 12.4918149 , 12.15529589, 11.6806499 , 12.45568681,
       11.87915744, 11.88642335, 12.18326969, 12.71006398, 11.6575314 ,
       11.69983683, 12.28454454, 11.60788088, 12.76949929, 11.77187522,
       11.75069362, 11.52693137, 11.79831104, 13.06741845, 11.86796058,
       11.66402938, 12.24086274, 11.50686937, 12.63603073, 11.95909202,
       12.3738389 , 12.22981346, 11.94914702, 11.71975077, 11.52755609,
       11.22541211, 12.0384418 , 12.63258942, 12.44913052, 12.62429556,
       12.21262205, 11.67359252, 12.6498914 , 11.9538629 , 12.05

In [294]:
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error (MSE) on test set:", mse)
print("R-squared score on test set:", r2)

Mean Squared Error (MSE) on test set: 0.024282014358500293
R-squared score on test set: 0.8568648304745705


In [295]:
intercept = linear_regression.intercept_
coefficients = linear_regression.coef_
print("Intercept:", intercept)
print("Coefficients:", coefficients)

Intercept: 0.9323760535682055
Coefficients: [-1.47854241e-06  5.11805450e-05  2.25066674e-05  8.12649960e-02
  2.23908880e-03  1.19944298e-03  1.77415371e-05  3.44752496e-03
  7.39199764e-03 -3.98769626e-05  1.25122320e-01  1.83171053e-02
  7.55158760e-05 -3.71587165e-03  3.30304649e-01  2.26634821e-02
  5.06551609e-03  1.61648138e-02  1.47297810e-02  8.28056844e-03
 -7.36107859e-02  6.20375652e-04  1.88499046e-02 -2.86968227e-04
  2.09087382e-02  1.18304004e-04  2.93213023e-03  3.76311886e-03
  4.34877368e-03  7.86523350e-03  8.66008433e-03  1.85952866e-02
 -4.35334725e-03 -5.82277724e-04 -1.09879239e-03  4.19873678e-01
  3.84210355e-01  3.56846067e-01  3.33688917e-01  2.15476696e-02
  4.49376964e-02  6.85071067e-03  3.11515181e-02  1.39081149e-02
 -6.03423279e-02 -1.33104197e-01 -1.24155984e-02 -4.72340826e-02
 -1.99021046e-02 -2.93725113e-02 -3.92102470e-02 -5.60034466e-02
  5.59825973e-02 -1.19748590e-01 -4.89363496e-02 -1.10365571e-01
 -1.47995843e-01 -7.77083633e-02 -8.77281126e-

In [296]:
predictions = linear_regression.predict(X)
r2 = r2_score(y, predictions)
print("R-squared score:", r2)

R-squared score: 0.921909227700066


## svm regression

In [297]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [298]:
from sklearn.svm import SVR
svm_regression = SVR()

In [299]:
svm_regression.fit(X_train, y_train)

SVR()

In [300]:
y_pred = svm_regression.predict(X_test)

In [301]:
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error (MSE) on test set:", mse)
print("R-squared score on test set:", r2)

Mean Squared Error (MSE) on test set: 0.06983288204850825
R-squared score on test set: 0.5883561691839845
